<a href="https://colab.research.google.com/github/rafaeldsouza/mestrado-ReconhecimentoPadrao/blob/main/ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Aluno:** Rafael de Souza Conceição

**Matéria:**Reconhecimento de Padrões - Francisco de Assis Boldt

Em um Jupyter Notebook:

Use uma base de dados para classificação com pelo menos 1000 amostras;
Escolha pelo menos três algoritmos de classificação;
Combine os classificadores de duas formas diferentes:
Voting
Stacking
Use gridsearch (ou randomsearch) para ajustar tantos os classificadores fracos quanto o ensemble;
Coloque o Jupyter Notebook no seu GitHub.
A entrega da tarefa é um link para o Jupyter Notebook no seu GitHub.

In [1]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.ensemble import StackingClassifier

In [2]:
dataset = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X, y = dataset.data, dataset.target
X.shape, y.shape

((1288, 1850), (1288,))

In [3]:
df = pd.DataFrame(np.c_[X,y])
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1811,1812,1813,1814,1815,1816,1817,1818,1819,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850
0,254.000000,254.000000,251.666672,240.333328,185.333328,144.000000,174.000000,196.666672,196.000000,192.333328,193.333328,191.666672,187.666672,185.000000,181.000000,179.000000,181.000000,180.000000,178.333328,174.666672,171.666672,167.666672,166.000000,163.666672,160.666672,154.666672,150.000000,148.333328,146.333328,145.666672,139.333328,129.666672,111.333336,89.666664,65.333336,50.666668,40.333332,253.333328,251.333328,247.333328,...,92.000000,86.333336,59.333332,44.333332,32.333332,26.666666,27.000000,26.333334,30.666666,109.333336,148.333328,161.000000,161.666672,150.666672,145.333328,143.000000,146.333328,147.000000,143.000000,141.333328,139.0,134.666672,126.666664,120.666664,114.666664,109.000000,103.000000,97.666664,100.333336,102.333336,100.000000,99.000000,95.666664,92.333336,90.666664,89.666664,87.333336,88.666664,86.666664,5.0
1,39.666668,50.333332,47.000000,54.666668,99.000000,120.666664,139.666672,157.666672,171.000000,177.666672,184.000000,189.333328,194.666672,200.666672,201.000000,201.000000,197.666672,194.000000,193.666672,191.333328,188.666672,185.333328,182.333328,177.666672,171.000000,164.333328,159.333328,161.666672,161.333328,152.666672,132.666672,108.333336,83.666664,65.000000,61.333332,51.000000,38.666668,47.666668,63.000000,65.333336,...,116.000000,116.333336,77.333336,76.333336,75.666664,78.333336,77.666664,74.000000,59.333332,30.333334,24.666666,84.000000,71.000000,44.000000,46.666668,56.000000,61.666668,67.000000,67.333336,69.000000,64.0,58.000000,51.666668,53.333332,54.000000,50.666668,43.666668,41.000000,40.000000,44.666668,59.666668,73.333336,77.666664,83.666664,103.333336,123.000000,117.666664,115.000000,133.666672,6.0
2,89.333336,104.000000,126.000000,141.333328,152.000000,155.333328,155.333328,160.000000,163.000000,166.666672,174.666672,179.666672,182.333328,185.666672,188.666672,193.333328,193.666672,193.333328,195.000000,196.333328,196.333328,197.333328,197.666672,197.666672,194.666672,191.333328,187.000000,186.666672,184.000000,169.000000,147.000000,166.000000,172.000000,159.666672,150.000000,150.333328,149.000000,100.000000,128.000000,143.666672,...,182.000000,181.333328,78.666664,66.000000,65.666664,67.000000,57.000000,74.666664,95.000000,57.666668,87.333336,92.333336,109.333336,132.666672,79.000000,66.666664,47.666668,91.000000,82.666664,89.000000,91.0,100.333336,111.666664,116.333336,118.666664,119.000000,123.333336,128.333328,129.666672,125.000000,111.666664,100.333336,106.666664,125.000000,117.666664,125.000000,175.333328,183.333328,183.000000,3.0
3,16.666666,7.666667,7.000000,6.000000,16.333334,70.000000,170.000000,169.666672,161.000000,106.333336,97.666664,141.333328,118.000000,101.333336,111.666664,135.000000,157.666672,148.000000,144.666672,152.666672,171.000000,182.000000,177.333328,163.333328,126.000000,106.333336,158.666672,185.000000,183.333328,177.000000,130.666672,39.333332,9.666667,7.000000,7.000000,6.333333,5.666667,70.000000,36.666668,14.000000,...,42.333332,55.666668,70.333336,63.333332,59.000000,57.000000,48.666668,37.666668,32.333332,26.000000,52.333332,184.000000,227.666672,224.333328,214.000000,193.000000,156.000000,105.333336,73.666664,66.333336,64.0,61.333332,63.333332,72.000000,84.333336,72.666664,70.333336,78.666664,90.666664,126.000000,190.000000,230.000000,238.000000,239.333328,233.666672,134.000000,27.666666,38.000000,51.333332,1.0
4,122.666664,121.000000,126.666664,129.333328,129.333328,134.666672,142.000000,142.666672,147.333328,152.000000,153.666672,157.666672,166.000000,169.333328,165.333328,164.333328,166.666672,164.333328,160.666672,156.666672,155.000000,151.333328,147.666672,146.000000,141.333328,138.333328,136.000000,131.000000,119.000000,10

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1288 entries, 0 to 1287
Columns: 1851 entries, 0 to 1850
dtypes: float64(1851)
memory usage: 18.2 MB


In [4]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, random_state=42)
X_tr.shape, X_te.shape, y_tr.shape, y_te.shape

((966, 1850), (322, 1850), (966,), (322,))

### RandomForest

In [35]:
modelo = RandomForestClassifier(random_state=42,max_features=0.30)
modelo.fit(X_tr, y_tr)
rf_pr = modelo.predict(X_te)
rfhits = rf_pr == y_te
sum(rfhits)/len(rfhits)

0.7204968944099379

### KNeighbors


In [44]:
modelo = KNeighborsClassifier(n_neighbors=9)
modelo.fit(X_tr, y_tr)
knn_pr = modelo.predict(X_te)
knnhits = knn_pr == y_te
sum(knnhits)/len(knnhits)

0.6211180124223602

### GaussianNB

In [13]:
modelo = GaussianNB()
modelo.fit(X_tr, y_tr)
gnb_pr = modelo.predict(X_te)
gnbhits = gnb_pr == y_te
sum(gnbhits)/len(gnbhits)

0.41304347826086957

### DecisionTree

In [46]:
model_tree = DecisionTreeClassifier(random_state=42)
model_tree.fit(X_tr, y_tr)
tree_pr = model_tree.predict(X_te)
treehits = tree_pr == y_te
sum(treehits)/len(treehits)

0.5248447204968945

### XGB

In [27]:
modelo = XGBClassifier(use_label_encoder=False, random_state=42)
modelo.fit(X_tr, y_tr)
xgb_pr = modelo.predict(X_te)
xgbhits = xgb_pr == y_te
xgbhits, sum(xgbhits)/len(xgbhits)

(array([ True,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True, False,  True, False,  True,  True, False,
         True,  True,  True, False,  True,  True,  True,  True,  True,
         True, False,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True, False,  True,  True,  True,  True,  True,
        False,  True, False,  True,  True,  True,  True,  True, False,
        False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True, False,  True, False,  True,  True,
        False, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True, False,  True, False,  True,  True, False,
         True,  True, False,  True,  True,  True,  True,  True,  True,
         True, False, False,  True, False,  True,  True, False,  True,
        False,  True,  True,  True,  True,  True,  True,  True,  True,
      

## Combinação de classificadores

### Voting

In [5]:

voting = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('xgb', XGBClassifier()),
    ('randomforest', RandomForestClassifier())
])
    
voting_params = {
    'knn__n_neighbors': [9],
    'xgb__use_label_encoder':[False],
    'xgb__random_state':[42],
    'randomforest__max_features':[0.30],
    'randomforest__random_state': [42]
}

modelo = GridSearchCV(voting, param_grid=voting_params, cv=3,scoring="accuracy")
modelo.fit(X_tr, y_tr)
voting_pr = modelo.predict(X_te)
votinghits = voting_pr == y_te
sum(votinghits)/len(votinghits)

0.7267080745341615

### Stacking

In [6]:
stack = StackingClassifier([
    ('knn', KNeighborsClassifier()),
    ('xgb', XGBClassifier()),
    ('randomforest', RandomForestClassifier())
], cv=3, passthrough=True)

stack_params = {
    'knn__n_neighbors': [9],
    'xgb__use_label_encoder':[False],
    'xgb__random_state':[42],
    'randomforest__max_features':[0.30],
    'randomforest__random_state': [42]
}

modelo = GridSearchCV(stack, param_grid=stack_params, cv=3,scoring="accuracy")

modelo.fit(X_tr, y_tr) 
stack_pr = modelo.predict(X_te)
stackhits = stack_pr == y_te
sum(stackhits)/len(stackhits)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.8043478260869565